In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
driver=webdriver.Chrome('chromedriver')

In [3]:
driver.get('https://fr.indeed.com/cmp/Soci%C3%A9t%C3%A9-G%C3%A9n%C3%A9rale-1/reviews')

In [6]:
from selenium.webdriver.remote.errorhandler import NoSuchElementException
fichier = open("sg.txt", "w", encoding="utf-8")
i = 1

suivant_xpath = '//*[@id="cmp-container"]/div/div[1]/main/div[2]/div[1]/div/nav/ul/li/a'

while(True):
    xpath = '//*[@id="cmp-container"]/div/div[1]/main/div[2]/div[1]/div/div[' + str(i) + ']'
    try :
        metier = driver.find_element(By.XPATH, xpath)
        print(metier.text)
        fichier.write(metier.text+ 2*'\n')
        sep = 10*'#'
        print(10*'#')
        fichier.write(sep + 2*'\n')
        i=i+1
        
    except NoSuchElementException : 
        time.sleep(60)
        suivant = driver.find_element(By.XPATH, suivant_xpath)
        suivant_xpath = '//*[@id="cmp-container"]/div/div[1]/main/div[2]/div[1]/div/nav/ul/li[2]/a'
        suivant.click()
        i=1


3,0
Cdd
Conseillère clientèle (ancien employé) - Massy (91) - 17 mai 2022
Avis à la une
L'avis le plus utile, sélectionné par Indeed
Bonne équipe, dommage que la direction soit bloquée dans le passé
Avez-vous trouvé cet avis utile ?
Oui
1
Non
##########
5,0
Une équipe au top
Responsable (ancien employé) - Falaise - 28 novembre 2022
Équipe au top, l'esprit d'équipe fait la force de cette agence.
Une directrice très à l 'écoute et humaine.
Une banque qui essaie de prendre soin de ses salariés
Avez-vous trouvé cet avis utile ?
Oui
Non
##########
Informations sur Société Générale
Sur la base de 196 réponses à l'enquête
Points forts
Capacité à apprendre de nouvelles choses
Capacité à atteindre des objectifs personnels
Sentiment d'appréciation personnelle
Points à améliorer
Rémunération juste par rapport au travail effectué
Soutien du responsable
Pas de pression
##########
4,0
agréable
Compliance Officer (ancien employé) - Nanterre (92) - 25 novembre 2022
bonne ambiance, bonne équipe, bon re

Informations sur Société Générale
Sur la base de 196 réponses à l'enquête
Points forts
Capacité à apprendre de nouvelles choses
Capacité à atteindre des objectifs personnels
Sentiment d'appréciation personnelle
Points à améliorer
Rémunération juste par rapport au travail effectué
Soutien du responsable
Pas de pression
##########
4,0
Management très agréable
Chargée d'accuiel clientle (ancien employé) - Paris 16e (75) - 19 octobre 2022
Plutôt bien dans l'ensemble avec des managers très à l'écoute
Avez-vous trouvé cet avis utile ?
Oui
Non
##########
4,0
Polyvalence rémunération faible
Chargée de clientèle particuliers (ancien employé) - Paris (75) - 18 octobre 2022
Peu d'évolution de carrière vie en agence difficile
Points positifs
Avantages sociaux
Points négatifs
Salaire faible
Évaluez votre entreprise pour afficher tous les avis
##########
3,0
L'ambiance compense le salaire
Analyste Crédit (ancien employé) - Île-de-France - 12 octobre 2022
Très bonne ambiance de travail dans l'ensembl

5,0
esprit d'équipe
Chargée de comptes (employé actuel) - Fontenay-sous-Bois (94) - 23 août 2022
bonne expérience
bonne opportunité
humain
professionnel
grand groupe
ya de la place pour tr le monde
Évaluez votre entreprise pour afficher tous les avis
##########
Donneriez-vous cinq étoiles à pôle Diversité et Réussite ?
Notez votre employeur maintenant pour aider d'autres personnes à faire de meilleurs choix de carrière.
##########
1,0
Je ne recommande pas …
Juriste Droit Bancaire & Paiement (ancien employé) - Paris (75) - 19 août 2022
… si vous êtes pro, que vous avez la volonté de préserver les intérêts de votre entreprise et que « l’humain » a de l’importance pour vous.
Évaluez votre entreprise pour afficher tous les avis
##########
1,0
crc de nanterre UNE HONTE
Conseillère clientèle (ancien employé) - Nanterre (92) - 11 août 2022
L'un des pires endroits pour travailler ! ma pire experience, j'ai quitté le crc de nanterre en 2020 avec un burn out et une depression.
Évaluez votre entr

2,0
l'aspect le plus difficile du poste
Responsable adjointe (ancien employé) - Val-d'Oise - 19 juin 2022
Hypocrisie des managers sur les primes aucunes transparences manipulateur hors pairs.
Points positifs
Rien
Points négatifs
Tout
Évaluez votre entreprise pour afficher tous les avis
##########
4,0
Équilibré
Conseiller en assurance (employé actuel) - Orléans (45) - 15 juin 2022
Il faut savoir utiliser ses compétences et les mettre en avant.
Certains manager sont excellent et à l’écoute.
Avec du travail vous pouvez faire de grandes choses et avoir des responsabilités
Mais le salaire n’est pas forcément représentatif des compétences
Évaluez votre entreprise pour afficher tous les avis
##########
4,0
Travail toujours en flux tendu
Conseiller clientèle premium (employé actuel) - Île-de-France - 15 juin 2022
Secteur d'activité toujours en manque de personnel, les formations arrivent 5/6mois après l'arrivée, on doit avoir beaucoup d'autonomie pour la prise de poste et connaître le metier.


4,0
Correct
Assistant Ressources Humaines (ancien employé) - Val de Fontenay (94) - 15 mai 2022
Bien correct .
Bonne accueil, je recommande. Forme bien .
Évaluez votre entreprise pour afficher tous les avis
##########
3,0
Très stressant pas assez d'accompagnement
Conseillère bancaire (employé actuel) - Paris (75) - 12 mai 2022
Un poste très stressant qui demande beaucoup d'organisation dès le départ. Énormément d'administratif et jamais le temps de faire la moindre chose. Je gère 2 portefeuilles clients alors que ça fait que 3 mois que je suis arrivée dans l'entreprise et sans aucune expérience en banque pas assez d'accompagnement.

Cependant l'équipe de mon agence est absolument superbe, malheureusement ils sont également très occupés et de ce fait je ne suis pas bien accompagnée, je suis lâchée dans la jungle, je dois improviser

Par contre tous les jours je suis en stress total avec la boule au ventre
Points positifs
Bonne ambiance
Points négatifs
Stress permanent
Évaluez votre entr

...

Informations sur Société Générale
Sur la base de 196 réponses à l'enquête
Points forts
Capacité à apprendre de nouvelles choses
Capacité à atteindre des objectifs personnels
Sentiment d'appréciation personnelle
Points à améliorer
Rémunération juste par rapport au travail effectué
Soutien du responsable
Pas de pression
##########
4,0
Environnement de travail agréable
Assistante commerciale de fin d'étude (ancien employé) - Massy, A8 - 12 juin 2013
Assistante commerciale de fin d’étude à la Société Générale à Massy (91)
 Assister un chargé de clientèle professionnel et privé
 Télémarketing
 Mise à jour des dossiers clients
Avez-vous trouvé cet avis utile ?
Oui
Non
2
##########
3,0
Une grande entreprise française
Chef de projet formation (employé actuel) - Paris La Défense - 11 juin 2013
La société a une culture d'entreprise très forte, très "à la française" dans les métiers traditionnels de banque de détail et dans ses fonctions support. Bien payé, bien encadré, vous bénéficiez à la S

3,0
Emploi Formateur
Conseiller clientèle Multimédia (ancien employé) - Centre de relation clientèle multimédia (CRCM) - 3 mars 2013
Journée de travail de 8h30 à 18h, c'est plus de 100 appels par jours, rythmes difficiles, mais ambiance de travail très agréable.

Les opportunités, au bout de 3 ans, étaient de rentrer en poste en agence.

L'aspect le plus difficile : Traiter les appels et répondre à la demande du client en 3min40 et rebondir sur le bon produit, celui qui répond au mieux à ses besoins et ses attentes

L'aspect le plus agréable: Très bonne ambiance de travail, et également l'aspect commercial du travail
Points positifs
ticket restaurant, ce, salaire intéressent
Points négatifs
durée des appels, pas de contact direct avec la clientèle
Évaluez votre entreprise pour afficher tous les avis
##########
5,0
Environnement de travail challegeant
RESPONSABLE PROJET INTERNATIONAL (employé actuel) - La défense - 22 février 2013
- projets de grande envergure
- environnement internatio

5,0
La vie Société Générale
chargée d'accueil (employé actuel) - Paris - 29 septembre 2012
L'esprit d'équipe est au coeur de notre quotidien et c'est ce que j'apprécie grandement.
Les opportunités d'évolution sont nombreuses et font que le travail est motivant.
Points positifs
polyvalence et esprit d'équipe
Évaluez votre entreprise pour afficher tous les avis
##########
5,0
Entreprise extraordianaire.
Agent administratif (ancien employé) - ORLEANS - 16 septembre 2012
Le poste en lui même était varié. En ce qui me concerne, je ne pouvais poursuivre dans cette entreprise du fait de notre départ de la région Orléanaise. Mes responsables ainsi que mes collègues étaient toujours présents pour une aide si besoin. Une solidarité y est présente au quotidien.
Points positifs
le poste d'administratif ainsi que les collègues
Points négatifs
il s'agissait d'une mission intérim.
Évaluez votre entreprise pour afficher tous les avis
##########
4,0
Avis d'intérimaire
Agent administratif (ancien employ

##########
Soumettre un avis
##########


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="cmp-container"]/div/div[1]/main/div[2]/div[1]/div/nav/ul/li[2]/a"}
  (Session info: chrome=107.0.5304.123)
Stacktrace:
Backtrace:
	Ordinal0 [0x007CACD3+2075859]
	Ordinal0 [0x0075EE61+1633889]
	Ordinal0 [0x0065B7BD+571325]
	Ordinal0 [0x0068AC2F+764975]
	Ordinal0 [0x0068AE1B+765467]
	Ordinal0 [0x006BD0F2+970994]
	Ordinal0 [0x006A7364+881508]
	Ordinal0 [0x006BB56A+963946]
	Ordinal0 [0x006A7136+880950]
	Ordinal0 [0x0067FEFD+720637]
	Ordinal0 [0x00680F3F+724799]
	GetHandleVerifier [0x00A7EED2+2769538]
	GetHandleVerifier [0x00A70D95+2711877]
	GetHandleVerifier [0x0085A03A+521194]
	GetHandleVerifier [0x00858DA0+516432]
	Ordinal0 [0x0076682C+1665068]
	Ordinal0 [0x0076B128+1683752]
	Ordinal0 [0x0076B215+1683989]
	Ordinal0 [0x00776484+1729668]
	BaseThreadInitThunk [0x7612FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x76FB7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x76FB7B8E+238]
